# Bank Heist Problem - Neil A

Imports

In [1]:
import numpy as np
import pandas as pd
import time
import math

Load data

In [39]:
df = pd.read_csv(r"C:\Users\neila\Desktop\Data Science\ds-algorithm-project-1-main\data\bank_data.csv")

Data Exploration

In [56]:
df.describe()

,id,x_coordinate,y_coordinate,money,time (hr)
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4999.50000,-0.016431,0.037561,18430.320000,0.750245
std,2886.89568,2.889568,2.881789,25382.131155,0.433541
min,0.00000,-4.999292,-4.999928,100.000000,0.000032
25%,2499.75000,-2.513854,-2.415033,900.000000,0.373141
50%,4999.50000,-0.024904,0.010442,5800.000000,0.749218
75%,7499.25000,2.461754,2.540303,26900.000000,1.122812
max,9999.00000,4.999851,4.999626,102300.000000,1.499851


In [57]:
df.head(10)

,id,x_coordinate,y_coordinate,money,time (hr)
0,0,2.851925,1.201712,29700,0.273403
1,1,4.150372,-2.654334,6500,0.064040
2,2,-1.494092,-1.230419,89400,0.127458
3,3,1.271326,-0.088520,96100,1.315029
4,4,2.471113,-0.592810,41100,0.164393
5,5,-3.169726,-1.965427,36300,1.469407
6,6,1.937701,-4.451860,100,1.303451
7,7,4.799624,-2.330932,6200,1.421906
8,8,1.747491,-1.791918,200,1.129879
9,9,-1.188827,1.067312,1500,1.014085


Solution

In [63]:
def bank_heist(data, time_limit):
    # Initialize all variables
    start_time = time.time()
    current_time = 0
    total_money = 0
    current_location = np.array([0, 0])
    banks_to_rob = []

    # Run until time limit reached or all banks visited
    while current_time <= time_limit and not data.empty:
        
        # Find most efficient bank to rob using money / time + travel time
        data['distance'] = np.sqrt((data['x_coordinate'] - current_location[0]) ** 2 +
                                   (data['y_coordinate'] - current_location[1]) ** 2)
        data['dynamic_value_ratio'] = data['money'] / (data['time (hr)'] + data['distance'] / 30)

        # Choose bank with highest dynamic value ratio
        chosen_bank = data.loc[data['dynamic_value_ratio'].idxmax()]

        # If we have time to get to and rob the bank, proceed and update all variables
        if current_time + chosen_bank['time (hr)'] + chosen_bank['distance'] / 30 <= time_limit:
            banks_to_rob.append(int(chosen_bank['id']))
            current_time += chosen_bank['time (hr)'] + chosen_bank['distance'] / 30
            total_money += chosen_bank['money']
            current_location = np.array([chosen_bank['x_coordinate'], chosen_bank['y_coordinate']])

        # Remove chosen bank from remaining data
        data = data[data['id'] != chosen_bank['id']]

    # When time is up or all banks visited, calculate execution time
    end_time = time.time()
    execution_time = end_time - start_time

    # Save results (with bank IDs reversed since we started at the escape zone)
    result = {
        'Banks to Rob': banks_to_rob[::-1],
        'Total Money': total_money,
        'Execution Time': execution_time,
        'Total Time Taken': current_time
    }

    return result

# Run code ------------------------------
# Use a copy of df to avoid modifying the original data
heist_result = bank_heist(df.copy(), 24)
for key, value in heist_result.items():
    print(f"{key}: {value}")
    
    

# Potential Impovements ---------
# Segment code into smaller functions for modularity
# Find ways to use more than 19 seconds to refine result

Banks to Rob: [4601, 3237, 2270, 3773, 5182, 2334, 2444, 8887, 1723, 5892, 5200, 2590, 5836, 5572, 4794, 3665, 4629, 6266, 5327, 1081, 2463, 6295, 5440, 8477, 9804, 5631, 5377, 5399, 1879, 8841, 1398, 5184, 4345, 9120, 757, 1860, 2344, 3025, 4293, 1844, 839, 4422, 3340, 3971, 1966, 359, 754, 6375, 7907, 670, 9469, 9275, 5381, 7769, 7625, 8909, 1397, 6156, 5126, 6987, 2458, 6478, 9290, 9928, 7649, 8169, 6281, 3136, 1599, 1193, 8849, 9908, 2729, 7764, 7701, 4036, 8503, 2827, 8407, 3919, 1997, 7531, 5356, 1447, 2028, 4362, 8829, 5474, 2643, 2521, 3239, 1676, 4742, 4983, 8288, 1976, 2243, 9779, 8125, 279, 6169, 2162, 8949, 6339, 3585, 4494, 5944, 2472, 70, 790, 1684, 6934, 488, 2226, 9880, 5563, 2639, 7494, 5971, 1455, 3605, 8375, 5719, 613, 6759, 781, 4234, 3186, 6216, 4757, 4605, 7877, 7689, 8355, 1757, 8908, 7265, 5622, 6317, 4610, 5627, 7801, 2, 58, 1372, 951, 9881, 7595, 7772, 507, 5298, 6022, 2626, 9640, 8846, 8231, 5166, 7665, 3089, 4807, 9228, 524, 9583, 3005, 7583, 444, 865, 3466,

Explanation

After initializing the variables and checking base cases (time and banks), I find the most efficient bank to rob using a ratio of money over time (time to rob + travel time), which is updated after
every robbery to reflect the travel time to each bank based on our current location (the coordinates 
of the previously robbed bank). Then if I have enough time, I rob the bank with the best ratio and
update my variables to reflect the time used and the money stolen, as well as my current location.

In order to simplify getting to the escape zone, I used (0,0) as my starting location and then
reversed my list of robbed banks. This allowed me to optimize my code by not having to include
and exception for the the last bank visited or calculate the distance from each bank to the
escape zone.

Results

Total Money: 14,159,300.0 $

Banks to Rob in Order: [4601, 3237, 2270, 3773, 5182, 2334, 2444, 8887, 1723, 5892, 5200, 2590, 5836, 5572,
               4794, 3665, 4629, 6266, 5327, 1081, 2463, 6295, 5440, 8477, 9804, 5631, 5377, 5399,
               1879, 8841, 1398, 5184, 4345, 9120, 757, 1860, 2344, 3025, 4293, 1844, 839, 4422,
               3340, 3971, 1966, 359, 754, 6375, 7907, 670, 9469, 9275, 5381, 7769, 7625, 8909, 1397,
               6156, 5126, 6987, 2458, 6478, 9290, 9928, 7649, 8169, 6281, 3136, 1599, 1193, 8849,
               9908, 2729, 7764, 7701, 4036, 8503, 2827, 8407, 3919, 1997, 7531, 5356, 1447, 2028,
               4362, 8829, 5474, 2643, 2521, 3239, 1676, 4742, 4983, 8288, 1976, 2243, 9779, 8125,
               279, 6169, 2162, 8949, 6339, 3585, 4494, 5944, 2472, 70, 790, 1684, 6934, 488, 2226,
               9880, 5563, 2639, 7494, 5971, 1455, 3605,8375, 5719, 613, 6759, 781, 4234, 3186, 6216,
               4757, 4605, 7877, 7689, 8355, 1757, 8908, 7265, 5622, 6317, 4610, 5627, 7801, 2, 58,
               1372, 951, 9881, 7595, 7772, 507, 5298, 6022, 2626, 9640, 8846, 8231, 5166, 7665, 3089,
               4807, 9228, 524, 9583, 3005, 7583, 444, 865, 3466, 3803, 8022, 6712, 2037, 1914, 5295,
               7343, 8703, 299, 4762, 6468, 3193, 8525, 7560, 5933, 487, 2331, 4499, 8690, 6528, 4287,
               9653, 6104, 5562, 3516, 4906, 7087, 517, 7074, 7258, 6740, 3026, 4696, 4789, 8436,
               7064, 6097, 8287, 3798, 4987, 9195, 5610, 433, 6254, 557, 3914, 8562, 2190, 6535, 4492,
               4627, 5155, 9736, 2656, 8206, 3297, 9401, 8966, 9049, 2741, 2346, 8469, 5296, 9378,
               2928, 9241, 7544, 3613, 9546, 5135]

Total Time Taken: 23.955108378954346 Hours

Execution Time: 19.279924154281616 Seconds